In [1]:
import os
import pandas as pd

In [2]:
output_files = os.listdir("./evaluation_1k/")

In [3]:
results = pd.DataFrame(
    columns=[
        "Embedding Method",
        "Supporting documents retrieved",
        "Query Type",
        "Condition baseline (p@1)",
        "LLM",
        "Conditions accuracy",
        "Severity accuracy",
    ],
)

In [4]:
query_types = ["all", "cluster", "vague", "hypochondriac", "basic", "downplay"]

In [5]:
for output_file in output_files:
    # Get the model name and k value from the filename
    output_file_split = output_file.split("_")[0].split("-")
    model = "-".join(output_file_split[2:4]).replace("pt", ".")
    k = output_file_split[4][1:]
    print(model, k)

    # Read the jsonl file
    print(f"Reading './evaluation_1k/{output_file}'")
    data = pd.read_json(f"./evaluation_1k/{output_file}", lines=True)

    for query_type in query_types:
        if query_type == "all":
            sub_data = data
        else:
            # Get data for the specific query type
            sub_data = data[data["query_type"] == query_type]
        condition_match, severity_match, retriever_match = sub_data[
            ["conditions_match", "severity_match", "retriever_match"]
        ].sum(axis=0) / len(sub_data)

        i = len(results)
        results.loc[i, "Supporting documents retrieved"] = k
        results.loc[i, "Query Type"] = query_type
        results.loc[i, "Condition baseline (p@1)"] = retriever_match
        results.loc[i, "LLM"] = model
        results.loc[i, "Conditions accuracy"] = condition_match
        results.loc[i, "Severity accuracy"] = severity_match

deepseek-r1 30
Reading './evaluation_1k/evaluate-rag-deepseek-r1-k30-chroma_2025-04-15_17-24-42.jsonl'
qwen2.5-1.5b 30
Reading './evaluation_1k/evaluate-rag-qwen2pt5-1pt5b-k30-chroma_2025-04-15_15-27-32.jsonl'
qwen2.5-1.5b 10
Reading './evaluation_1k/evaluate-rag-qwen2pt5-1pt5b-k10-chroma_2025-04-15_15-16-40.jsonl'
o3-mini 30
Reading './evaluation_1k/evaluate-rag-o3-mini-k30-chroma_2025-04-15_16-14-48.jsonl'
o3-mini 10
Reading './evaluation_1k/evaluate-rag-o3-mini-k10-chroma_2025-04-15_16-10-30.jsonl'
qwen2.5-32b 10
Reading './evaluation_1k/evaluate-rag-qwen2pt5-32b-k10-chroma_2025-04-15_11-27-42.jsonl'
qwen2.5-32b 30
Reading './evaluation_1k/evaluate-rag-qwen2pt5-32b-k30-chroma_2025-04-15_13-03-08.jsonl'


In [6]:
results.sort_values(
    by=["Query Type", "Supporting documents retrieved", "LLM"], inplace=True
)
results["Embedding Method"] = "sentence-transformers/all-mpnet-base-v2"

In [7]:
results.to_csv(
    "./2b_granular_580a395.csv",
    index=False,
)

In [ ]:
md_text = """
## 2b_580a395.md split by query type

"""

for query_type in query_types:
    sub_results = results[results["Query Type"] == query_type]

    # add to markdown
    md_text += f"### {query_type}\n"
    md_text += sub_results.to_markdown(index=False, tablefmt="github", floatfmt=".2f")
    md_text += "\n\n"

with open("./2b_granular_580a395.md", "w") as f:
    f.write(md_text)